<span style="font-size:large;">
<div style="text-align: right">IM-UH 1511 <b>Introduction to Digital Humanities</b><br/>
Student name: <span style="color:blue"><b>X XX</b></span><br/>
Grade: <span style="color:red"><b></b></span><br/><br/></div>

## <span style="color:red">HOMEWORK 8</span>
# <span style="color:green">Tracking the global coronavirus outbreak
    
### <span style="color:blue">Please create a folder named "html_map_output" in the same place where you're running this notebook and then submit the zipped the contents of this folder together with your homework notebook.

In [1]:
# tracker
import COVID19Py # pip install COVID19Py
# https://pypi.org/project/COVID19Py/ 
# https://github.com/ExpDev07/coronavirus-tracker-api

# folium
import folium # pip install folium OR conda install -c conda-forge folium
from folium import plugins
import folium.plugins
from folium.plugins import MarkerCluster
MarkerCluster()
from folium.plugins import StripePattern

# essential libraries
import random
import datetime
from datetime import timedelta  
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import calmap
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
 
# html embedding
from IPython.display import Javascript
from IPython.core.display import display
from IPython.core.display import HTML

# warnings handling
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter('ignore')

In [2]:
covid19 = COVID19Py.COVID19()
locations = covid19.getLocations()
country=[]
country_code=[]
province=[]
latitude=[]
longitude=[]
confirmed=[]
deaths=[]
recovered=[]
last_updated=[]
for i in range(len(locations)):
    column=locations[i]
    country.append(column['country'])
    country_code.append(column['country_code'])
    province.append(column['province'])
    latitude.append(float(column['coordinates']['latitude']))
    longitude.append(float(column['coordinates']['longitude']))
    confirmed.append(int(column['latest']['confirmed']))
    deaths.append(int(column['latest']['deaths']))
    recovered.append(int(column['latest']['recovered']))
    last_updated.append(column['last_updated'])
df = pd.DataFrame(
    {'country':country,
     'country_code':country_code,
     'province':province,
     'latitude': latitude, 
     'longitude':longitude,
     'confirmed':confirmed,
     'deaths':deaths,
     'recovered':recovered,
     'last_updated':last_updated
    })
df=df[['country_code','country','province','latitude','longitude','confirmed','deaths','recovered','last_updated']]
df['last_updated'] = pd.to_datetime(df['last_updated'], format="%Y-%m-%d %H:%M:%S")
df

,country_code,country,province,latitude,longitude,confirmed,deaths,recovered,last_updated
0,AF,Afghanistan,,33.000000,65.000000,29157,598,0,2020-06-23 05:49:18.325311+00:00
1,AL,Albania,,41.153300,20.168300,1995,44,0,2020-06-23 05:49:18.331969+00:00
2,DZ,Algeria,,28.033900,1.659600,11920,852,0,2020-06-23 05:49:18.338375+00:00
3,AD,Andorra,,42.506300,1.521800,855,52,0,2020-06-23 05:49:18.345087+00:00
4,AO,Angola,,-11.202700,17.873900,186,10,0,2020-06-23 05:49:18.351936+00:00
...,...,...,...,...,...,...,...,...,...
261,ST,Sao Tome and Principe,,0.186360,6.613081,702,12,0,2020-06-23 05:49:20.127888+00:00
262,YE,Yemen,,15.552727,48.516388,967,257,0,2020-06-23 05:49:20.134243+00:00
263,KM,Comoros,,-11.645500,43.333300,247,5,0,2020-06-23 05:49:20.140706+00:00
264,TJ,Tajikistan,,38.861034,71.276093,5513,52,0,2020-06-23 05:49:20.147201+00:00


In [3]:
dfn=df.copy()
dfc=df.copy()
ccsd=sorted(set(dfn.country.tolist()))
dfn['confirmed'] = dfn['confirmed'].astype(int)
dfn['deaths'] = dfn['deaths'].astype(int)
dfn['recovered'] = dfn['recovered'].astype(int)
tconfirmed = dfn['confirmed'].sum()
tdeaths = dfn['deaths'].sum()
trecovered = dfn['recovered'].sum()
dtnow=datetime.datetime.now()
dtupd=[str(d)[:11] for d in dfn['last_updated'].tolist()][0]
print("Time now:", dtnow)
print("Time data upadated:",dtupd)
print(len(ccsd), "Countries")
print(tconfirmed,'confirmed')
print(tdeaths,'deaths')
# print(trecovered,'recovered')

Time now: 2020-06-23 10:08:24.241312
Time data upadated: 2020-06-23 
188 Countries
9098643 confirmed
472171 deaths


In [4]:
# cases 
cases = ['confirmed', 'deaths', 'recovered', 'active']

# Active Case = confirmed - deaths - recovered
df['active'] = df['confirmed'] - df['deaths'] - df['recovered']

# # replacing Mainland china with just China
# full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

# filling missing values 
df[['province']] = df[['province']].fillna('')
df[cases] = df[cases].fillna(0)

# fixing datatypes
df['recovered'] = df['recovered'].astype(int)

df.sample(6)

,country_code,country,province,latitude,longitude,confirmed,deaths,recovered,last_updated,active
209,TH,Thailand,,15.0000,101.0000,3151,58,0,2020-06-23 05:49:19.785046+00:00,3093
125,GY,Guyana,,5.0000,-58.7500,205,12,0,2020-06-23 05:49:19.239884+00:00,193
109,FR,France,Guadeloupe,16.2500,-61.5833,174,14,0,2020-06-23 05:49:19.136215+00:00,160
159,MD,Moldova,,47.4116,28.3699,14363,480,0,2020-06-23 05:49:19.460077+00:00,13883
70,CN,China,Ningxia,37.2692,106.1655,75,0,0,2020-06-23 05:49:18.880233+00:00,75
140,JO,Jordan,,31.2400,36.5100,1042,9,0,2020-06-23 05:49:19.337618+00:00,1033


In [5]:
# cases in the ships
ship = df[df['province'].str.contains('Grand Princess')|df['country'].str.contains('Diamond Princess')]

# china and the row
china = df[df['country']=='China']
row = df[df['country']!='China']

# latest
# df = df[full_table['Date'] == max(full_table['Date'])].reset_index()
china_latest = df[df['country']=='China']
row_latest = df[df['country']!='China']

# latest condensed
df_grouped = df.groupby('country')['confirmed', 'deaths', 'recovered', 'active'].sum().reset_index()
china_latest_grouped = china_latest.groupby('province')['confirmed', 'deaths', 'recovered', 'active'].sum().reset_index()
row_latest_grouped = row_latest.groupby('country')['confirmed', 'deaths', 'recovered', 'active'].sum().reset_index()

In [6]:
log_confirmed=np.log(df_grouped["confirmed"])
df_grouped["log_confirmed"]=log_confirmed

In [7]:
temp = df.groupby(['country', 'province'])['confirmed', 'deaths', 'recovered', 'active'].max()
temp['global_mortality'] = temp['deaths']/temp['confirmed']
temp['deaths_per_100_confirmed_cases'] = temp['global_mortality']*100
temp.style.background_gradient(cmap='Pastel1')
temp.sample(10)

,,confirmed,deaths,recovered,active,global_mortality,deaths_per_100_confirmed_cases
country,province,,,,,,
Algeria,,11920,852,0,11068,0.071477,7.147651
Canada,Grand Princess,13,0,0,13,0.000000,0.000000
US,,2312302,120402,0,2191900,0.052070,5.207019
China,Heilongjiang,947,13,0,934,0.013728,1.372756
France,New Caledonia,21,0,0,21,0.000000,0.000000
Malaysia,,8587,121,0,8466,0.014091,1.409107
Netherlands,"Bonaire, Sint Eustatius and Saba",7,0,0,7,0.000000,0.000000
Jamaica,,665,10,0,655,0.015038,1.503759
Ireland,,25383,1717,0,23666,0.067644,6.764370


In [8]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

temp_f = df_grouped.sort_values(by='confirmed', ascending=False)
temp_f = temp_f[['country', 'confirmed', 'active', 'deaths']] #, 'recovered'
temp_f = temp_f.reset_index(drop=True)

temp_f.style.background_gradient(cmap="Blues", subset=['confirmed', 'active'])\
            .background_gradient(cmap="Greens", subset=[])\
            .background_gradient(cmap="Reds", subset=['deaths'])

,country,confirmed,active,deaths
0,US,2312302,2191900,120402
1,Brazil,1106470,1055199,51271
2,Russia,591465,583269,8196
3,India,440215,426204,14011
4,United Kingdom,306761,264030,42731
5,Peru,257447,249224,8223
6,Chile,246963,242461,4502
7,Spain,246504,218180,28324
8,Italy,238720,204063,34657
9,Iran,207525,197783,9742


In [9]:
temp_flg = temp_f[temp_f['deaths']>0][['country', 'deaths']]
temp_flg['Deaths / 100 Cases'] = round((temp_f['deaths']/temp_f['confirmed'])*100, 2)
temp_flg.sort_values('deaths', ascending=False).reset_index(drop=True).style.background_gradient(cmap='Reds')

,country,deaths,Deaths / 100 Cases
0,US,120402,5.210000
1,Brazil,51271,4.630000
2,United Kingdom,42731,13.930000
3,Italy,34657,14.520000
4,France,29666,15.030000
5,Spain,28324,11.490000
6,Mexico,22584,12.200000
7,India,14011,3.180000
8,Iran,9742,4.690000
9,Belgium,9696,16.010000


In [10]:
# World wide

m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               zoom_start=1) #min_zoom=1, max_zoom=4, 

full_latest=df

for i in range(0, len(full_latest)):
    folium.Circle(
        location=[full_latest.iloc[i]['latitude'], full_latest.iloc[i]['longitude']],
        color='crimson', 
        tooltip =   '<li><bold>Country : '+str(full_latest.iloc[i]['country'])+
                    '<li><bold>Province : '+str(full_latest.iloc[i]['province'])+
                    '<li><bold>Confirmed : '+str(full_latest.iloc[i]['confirmed'])+
                    '<li><bold>Deaths : '+str(full_latest.iloc[i]['deaths']),
        radius=int(full_latest.iloc[i]['confirmed'])*0.7).add_to(m)  
m

In [11]:
def sdf(df):
    locations = covid19.getLocations()
    country=[]
    country_code=[]
    province=[]
    latitude=[]
    longitude=[]
    confirmed=[]
    deaths=[]
    recovered=[]
    last_updated=[]
    for i in range(len(locations)):
        column=locations[i]
        country.append(column['country'])
        country_code.append(column['country_code'])
        province.append(column['province'])
        latitude.append(float(column['coordinates']['latitude']))
        longitude.append(float(column['coordinates']['longitude']))
        confirmed.append(str(column['latest']['confirmed']))
        deaths.append(str(column['latest']['deaths']))
        recovered.append(str(column['latest']['recovered']))
        last_updated.append(column['last_updated'])
    df = pd.DataFrame(
        {'country':country,
         'country_code':country_code,
         'province':province,
         'latitude': latitude, 
         'longitude':longitude,
         'confirmed':confirmed,
         'deaths':deaths,
         'recovered':recovered,
         'last_updated':last_updated
        })
    df=df[['country_code','country','province','latitude','longitude','confirmed','deaths','recovered','last_updated']]
    df['last_updated'] = pd.to_datetime(df['last_updated'], format="%Y-%m-%d %H:%M:%S")
    return df

<h3 align="center" style="font-size:20px"><span style="color:red"><b>MANUAL INSERTION OF DATE BELOW!

In [12]:
dfc=sdf(dfc)
map_center = [dfc["latitude"].mean(), dfc["longitude"].mean()]
map_3 = folium.Map(location=[dfc['latitude'].mean(), 
                                dfc['longitude'].mean()], #tiles='CartoDB positron', 
                                zoom_start=1)

locations = dfc[['latitude', 'longitude']]
locationlist = locations.values.tolist()

mc = MarkerCluster().add_to(map_3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=dfc['country'][point]+' '+dfc['province'][point]+' Confirmed: '+dfc['confirmed'][point]+' Deaths: '+dfc['deaths'][point], icon=folium.Icon(color='crimson', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(mc)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Clustered Countries with Confirmed Cases on 2020-04-08</b></h3>
             '''
map_3.get_root().html.add_child(folium.Element(title_html))
    
map_3.save('html_map_output/'+dtupd+'_covid19locations_clusters.html')
map_3

In [13]:
# Confirmed
sst='Countries with Confirmed Cases on %s' %dtupd
fig = px.choropleth(df_grouped, locations="country", 
                    locationmode='country names', color="log_confirmed", #np.log(df_grouped["confirmed"]), #deaths", #np.log(df_grouped["confirmed"]), 
                    hover_data=['country','confirmed','deaths'], #hover_name="country", 
                    color_continuous_scale="Sunsetdark", 
                    title=sst, #Countries with Confirmed Cases'+'_dtupd', #2020-04-03',
#                     projection='natural earth',
                    width=800 #,height=800
                   )
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [14]:
import plotly

plotly.offline.plot(fig, filename='html_map_output/'+dtupd+'_covid19locations_choropleth.html')

'html_map_output/2020-06-23 _covid19locations_choropleth.html'